<a href="https://colab.research.google.com/github/MacWorldPro/Module_34/blob/main/Stacking_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load the dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize base models
base_model_1 = DecisionTreeClassifier(random_state=42)
base_model_2 = LogisticRegression(max_iter=200, random_state=42)
base_model_3 = KNeighborsClassifier()

# Train base models
base_model_1.fit(X_train, y_train)
base_model_2.fit(X_train, y_train)
base_model_3.fit(X_train, y_train)

# Generate out-of-fold predictions
base_model_1_oof = cross_val_predict(base_model_1, X_train, y_train, cv=5, method='predict_proba')
base_model_2_oof = cross_val_predict(base_model_2, X_train, y_train, cv=5, method='predict_proba')
base_model_3_oof = cross_val_predict(base_model_3, X_train, y_train, cv=5, method='predict_proba')

# Stack the predictions as features for the meta-model
meta_features = np.hstack((base_model_1_oof, base_model_2_oof, base_model_3_oof))

# Initialize and train meta-model
meta_model = LogisticRegression(max_iter=200, random_state=42)
meta_model.fit(meta_features, y_train)

# Generate predictions for the test set from base models
base_model_1_test = base_model_1.predict_proba(X_test)
base_model_2_test = base_model_2.predict_proba(X_test)
base_model_3_test = base_model_3.predict_proba(X_test)

# Stack the test set predictions
meta_test_features = np.hstack((base_model_1_test, base_model_2_test, base_model_3_test))

# Predict with the meta-model
meta_test_pred = meta_model.predict(meta_test_features)

# Evaluate the performance
print("Stacking Model Accuracy:", accuracy_score(y_test, meta_test_pred))
print("Classification Report:\n", classification_report(y_test, meta_test_pred))


Stacking Model Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

